In [83]:
import pandas as pd
import numpy as np
import re
import enchant
import nltk
import csv
#nltk.download()

In [84]:
##删除文件中的乱码
def delete(aaa,number):                       #传入原csv文件每行元素和元素个数
    bbb = []
    for i in range(0,number):                   #分别对每行中每个元素进行处理
        ccc = aaa[i].encode("ASCII","ignore")   #删除乱码，乱码没有对应的ASCII编码
        bbb.append(ccc.decode())                #将删除乱码后的字符串重新组成一个列表
    return bbb

csv_file1=csv.reader(open('complaint1700.csv','r'))    
csv_file=csv.writer(open('complaint_new.csv','a',newline=''),dialect='excel')
content=[] #用来存储整个文件的数据，存成一个列表，列表的每一个元素又是一个列表，表示的是文件的某一行

for line in csv_file1:
    number = len(line)
    #将整个csv文件处理后组成一个二维列表，该列表每个元素都是一个列表，对应原csv文件的每一行内容（已经删除乱码）
    content.append(delete(line,number)) 
    
for i in range(0,len(content)):
    csv_file.writerow(content[i])       #写进一个新的csv文件


In [85]:
##删除文件中的乱码
def delete(aaa,number):                       #传入原csv文件每行元素和元素个数
    bbb = []
    for i in range(0,number):                   #分别对每行中每个元素进行处理
        ccc = aaa[i].encode("ASCII","ignore")   #删除乱码，乱码没有对应的ASCII编码
        bbb.append(ccc.decode())                #将删除乱码后的字符串重新组成一个列表
    return bbb

csv_file2=csv.reader(open('noncomplaint1700.csv','r'))    
csv_file3=csv.writer(open('noncomplaint_new.csv','a',newline=''),dialect='excel')
content=[] #用来存储整个文件的数据，存成一个列表，列表的每一个元素又是一个列表，表示的是文件的某一行

for line in csv_file2:
    number = len(line)
    #将整个csv文件处理后组成一个二维列表，该列表每个元素都是一个列表，对应原csv文件的每一行内容（已经删除乱码）
    content.append(delete(line,number)) 
    
for i in range(0,len(content)):
    csv_file3.writerow(content[i])       #写进一个新的csv文件

In [86]:
##删除文件中的乱码
def delete(aaa,number):                       #传入原csv文件每行元素和元素个数
    bbb = []
    for i in range(0,number):                   #分别对每行中每个元素进行处理
        ccc = aaa[i].encode("ASCII","ignore")   #删除乱码，乱码没有对应的ASCII编码
        bbb.append(ccc.decode())                #将删除乱码后的字符串重新组成一个列表
    return bbb

csv_file4=csv.reader(open('testdata1.csv','r'))    
csv_file5=csv.writer(open('testdata_new.csv','a',newline=''),dialect='excel')
content=[] #用来存储整个文件的数据，存成一个列表，列表的每一个元素又是一个列表，表示的是文件的某一行

for line in csv_file4:
    number = len(line)
    #将整个csv文件处理后组成一个二维列表，该列表每个元素都是一个列表，对应原csv文件的每一行内容（已经删除乱码）
    content.append(delete(line,number)) 
    
for i in range(0,len(content)):
    csv_file5.writerow(content[i])       #写进一个新的csv文件

In [87]:
complaint=pd.read_csv('complaint_new.csv')
noncomplaint=pd.read_csv('noncomplaint_new.csv')
testdata=pd.read_csv('testdata_new.csv')

In [343]:
complaint1=complaint.copy()
noncomplaint1=noncomplaint.copy()
#将complain和noncomplain的comment垂直组合
comment=pd.concat([complaint1,noncomplaint1]).reset_index()
#设置label项目，抱怨为0，非抱怨为1
target1=np.zeros((complaint1.shape[0],1))
target2=np.ones((noncomplaint1.shape[0],1))
target=np.vstack((target1,target2))
comment['target']=target

In [344]:
comment1=comment.copy()
testdata1=testdata.copy()

In [345]:
#将所有的comment合为一个，便于处理
def text_join(data):
    com=data.tweet.tolist()
    a=''
    for i in range(len(com)):
        a=a+com[i]+' ¥'
    return a

In [346]:
#删除数据中的@部分、&部分和网址的算法
def drop_words(x):
    #运用re正则库
    a=re.compile(r'@\w*|\&\w*|http:\/\/\w*\.\w*\/\w*|\\')
    new_comment=a.sub('',x)
    return new_comment

In [347]:
#检查更正算法
#利用pyenchant文字检查包
def spellcheck(x):
    from enchant.checker import SpellChecker
    #设置为英文
    chkr = SpellChecker("en_US")
    chkr.set_text(x)
    #遍利x中的每一个字符
    for j in chkr:
        b=j.suggest()
        #if没有suggest的单词就不进行替换
        if len(b)==0:
            j.replace('')
        #如果有suggest的单词就以第一个进行替换
        else:
            j.replace(b[0])
    #获取替换后的文本
    new_comment1=chkr.get_text()
    return new_comment1

In [348]:
#缩略词还原
def cont_reback(x):
    #编写还原列表
    cont_patterns = [
        (b"[^a-zA-Z]u[^a-zA-Z]", b"you"),
        (b"[^a-zA-Z]r[^a-zA-Z]", b"are"),
        (b"[^a-zA-Z]y[^a-zA-Z]", b"why"),
        (b"[^a-zA-Z]b4[^a-zA-Z]", b"before"),
        (b'(W|w)on\'t', b'will not'),
        (b'(W|w)ouldn\'t', b'would not'),
        (b'(C|c)an\'t', b'can not'),
        (b'(I|i)\'m', b'i am'),
        (b'(A|a)in\'t', b'is not'),
        (b'(\w+)\'ll', b'\g<1> will'),
        (b'(\w+)n\'t', b'\g<1> not'),
        (b'(\w+)\'ve', b'\g<1> have'),
        (b'(\w+)\'s', b'\g<1> is'),
        (b'(\w+)\'re', b'\g<1> are'),
        (b'(\w+)\'d', b'\g<1> would'),
        (b'(W|w)tf', b'what the fuck')]
    #替代函数
    patterns = [(re.compile(regex), repl) for (regex, repl) in cont_patterns]
    #str转化为bytes形式
    c=bytes(x,encoding='utf8')
    for (pattern, repl) in patterns:
        #如果含有这一缩写就进行替换
        if pattern.search(c):
            c= re.sub(pattern, repl, c)
    #bytes转化为str形式
    new_comment2=str(c,encoding='utf8')
    return new_comment2

In [349]:
#词形还原
def wordlemmatize(x):
    #将句子拆分成单个单词
    d=str.split(x,' ')
    #运用nltk
    from nltk.stem.wordnet import WordNetLemmatizer
    from nltk import pos_tag #词性标签
    from nltk import word_tokenize#单词还本
    #设置单词和词性返回算法
    def lemmatize(token, tag):
        if tag[0].lower() in ['n', 'v']:
            return lemmatizer.lemmatize(token, tag[0].lower())
        return token
    #单词还原
    lemmatizer = WordNetLemmatizer()
    #单词还原的词性列表
    tagged_corpus = [pos_tag(word_tokenize(document)) for document in d]
    e=[]
    #遍历词性列表
    for i in tagged_corpus:
        for token, tag in i:
            f=lemmatize(token, tag)
            e.append(f)
    #重新转化为句子
    new_comment3=' '.join(e)
    return new_comment3

In [350]:
#去除标点符号和数字
def exclude_word(x):
    #设置要去除的标点符号
    exclude=re.compile(r'\.|\,|\;|\?|\$|\d|\\|\/|\\\\|\(|\)|\:|\_|\#')
    new_comment4=exclude.sub('',x)
    return new_comment4

In [351]:
#利用TF-IDF进行特征处理
def tfidf_demo(x):
    from sklearn.feature_extraction.text import TfidfVectorizer
    #引入停用词
    #from nltk.corpus import stopwords
    stopwords=['i','you','we','he','she',
               'be','have','they','just','one',
               'the','a','aa','aaa','will',
               'would','as','to','this','there',
               'can','could','of','for']
    tf_idf=TfidfVectorizer(stop_words=stopwords)
    tfidf_matrix = tf_idf.fit_transform(x)
    #返回特征值矩阵，特征值名称，tf_idf生成器
    return tfidf_matrix.toarray(),tf_idf.get_feature_names(),tf_idf

In [352]:
def data_preprocessing(data):
    comment=text_join(data)
    #Step1:删除数据中的@部分、&部分以及网址
    new_comment=drop_words(comment)
    #Step2:拼写检查更正
    new_comment1=spellcheck(new_comment)
    #Step3:缩略词还原
    new_comment2=cont_reback(new_comment1)
    #Step4:词形还原
    new_comment3=wordlemmatize(new_comment2)
    #Step5:转化为小写
    new_comment3=new_comment3.lower()                                                                                       
    #Step6:去除标点符号和数字
    new_comment4=exclude_word(new_comment3)
    comment_new=str.split(new_comment4,'¥')
    #Step7:特征处理
    feature_matrix,feature_names,tf_idf=tfidf_demo(comment_new)
    #返回特征值矩阵，特征值名称，tf_idf生成器
    return feature_matrix,feature_names,tf_idf

In [353]:
dataset,feature_names,tf_idf=data_preprocessing(comment1)

In [354]:
labelset=np.array(list(comment.target.values)+[1])

In [355]:
def Standard_demo(x):
    from sklearn.preprocessing import StandardScaler
    standard=StandardScaler(with_mean=False)
    standard_result=standard.fit_transform(x)
    return standard_result

In [356]:
def Minmax_demo(x):
    from sklearn.preprocessing import MinMaxScaler
    minmax=MinMaxScaler()
    minmax_result=minmax.fit_transform(x)
    return minmax_result

In [357]:
def PCA_demo(x):
    from sklearn.decomposition import PCA
    pca=PCA(n_components=0.95)
    x_pca=pca.fit_transform(x)
    return x_pca

## 多项式贝叶斯
1. 最大最小特征化处理
2. 划分训练集和测试集，选取75%作为训练集，25%作为测试，附加42个随机值
3. 朴素贝叶斯，拉普拉斯平滑系数为20

In [358]:
def MNB_demo():
    ##建立分析模型_朴素贝叶斯
    from sklearn.model_selection import  train_test_split
    from sklearn.naive_bayes import MultinomialNB
    x=dataset
    y=labelset
    x=Minmax_demo(x)
    X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=42)
    #使用朴素贝叶斯进行训练
    clf = MultinomialNB(alpha=20)   
    clf.fit(X_train,y_train)    # 利用训练数据对模型参数进行估计
    y_predict = clf.predict(X_test)     # 对参数进行预测
    #获取结果报告
    print('The Accuracy of Naive Bayes Classifier is:', clf.score(X_test,y_test))
    return clf


In [359]:
mnb=MNB_demo()

The Accuracy of Naive Bayes Classifier is: 0.7337278106508875


## 随机森林
1. 标准分布特征化处理
2. 运用主成分分析进行降维
3. 划分训练集和测试集，选取75%作为训练集，25%作为测试，附加42个随机值
4. 选取100个小决策树进行集成学习

In [360]:
def RandomForest_demo():
    from sklearn.model_selection import  train_test_split
    from sklearn.ensemble import RandomForestClassifier
    x=dataset
    y=labelset
    x=Standard_demo(x)
    x=PCA_demo(x)
    X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=42)
    #使用朴素贝叶斯进行训练
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(X_train,y_train)    # 利用训练数据对模型参数进行估计
    y_predict = clf.predict(X_test)     # 对参数进行预测
    #获取结果报告
    print('The Accuracy of RandomForest Classifier is:', clf.score(X_test,y_test))
    return clf


In [361]:
RandomForest=RandomForest_demo()

The Accuracy of RandomForest Classifier is: 0.6153846153846154


## KNN
1. 标准分布特征化处理
2. 运用主成分分析进行降维
3. 划分训练集和测试集，选取75%作为训练集，25%作为测试，附加42个随机值
4. 选取k=33（33个排名最靠前的邻居），采用蛮力实现（brute）的方法计算其距离，距离的加权方式为按照距离加权（distance）

In [362]:
def KNN_demo():
    ##建立分析模型_朴素贝叶斯
    from sklearn.model_selection import  train_test_split
    from sklearn.neighbors import KNeighborsClassifier
    x=dataset
    y=labelset
    x=Standard_demo(x)
    x=PCA_demo(x)
    X_train,X_test,y_train,y_test = train_test_split(dataset,labelset,test_size=0.25,random_state=42)
    clf=KNeighborsClassifier(n_neighbors=33,algorithm='brute',weights='distance')
    clf.fit(X_train,y_train)
    print('The Accuracy of KNN Classifier is:',clf.score(X_test,y_test))
    return clf

In [363]:
KNN=KNN_demo()

The Accuracy of KNN Classifier is: 0.7230769230769231


## 投票集成方法
1. 标准分布特征化处理
2. 运用主成分分析进行降维
3. 划分训练集和测试集，选取75%作为训练集，25%作为测试，附加42个随机值
4. 选取刚才训练好的三个分类器进行集成
5. 投票方式为软投票，即按照比例加权平均，朴素贝叶斯和KNN比重分别为3/7，随机森林比重为1/7

In [364]:
def vote_demo():
    from sklearn.ensemble import VotingClassifier
    from sklearn.model_selection import  train_test_split
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.naive_bayes import MultinomialNB
    
    x=dataset
    y=labelset
    x=Standard_demo(x)
    x=PCA_demo(x)
    X_train,X_test,y_train,y_test = train_test_split(dataset,labelset,test_size=0.25,random_state=42)
    clf=VotingClassifier(estimators=
                         [('mnb',mnb),
                          ('RandomForest',RandomForest),
                          ('KNN',KNN)],voting='soft',weights=[3,1,3]）
    clf.fit(X_train,y_train)
    
    print('The Accuracy of voting Classifier is:',clf.score(X_test,y_test))
    return clf

In [365]:
vote=vote_demo()

The Accuracy of voting Classifier is: 0.749112426035503


In [366]:
def test_demo():
    comment=text_join(testdata1)
    #Step1:删除数据中的@部分、&部分以及网址
    new_comment=drop_words(comment)
    #Step2:拼写检查更正
    new_comment1=spellcheck(new_comment)
    #Step3:缩略词还原
    new_comment2=cont_reback(new_comment1)
    #Step4:词形还原
    new_comment3=wordlemmatize(new_comment2)
    #Step5:转化为小写
    new_comment3=new_comment3.lower()                                                                                       
    #Step6:去除标点符号和数字
    new_comment4=exclude_word(new_comment3)
    comment_new=str.split(new_comment4,'¥')
    #Step7:特征处理
    test_tf_idf =tf_idf.transform(comment_new)
    x=test_tf_idf
    x=Standard_demo(x)
    prediction=vote.predict(x)
    return prediction


In [367]:
prediction=test_demo()

In [368]:
testdata1['prediction']=list(prediction)[0:-1]
testdata1=testdata1[testdata1['prediction']==1]

In [369]:
def test_demo():
    comment=text_join(testdata1)
    #Step1:删除数据中的@部分、&部分以及网址
    new_comment=drop_words(comment)
    #Step2:拼写检查更正
    new_comment1=spellcheck(new_comment)
    #Step3:缩略词还原
    new_comment2=cont_reback(new_comment1)
    #Step4:词形还原
    new_comment3=wordlemmatize(new_comment2)
    #Step5:转化为小写
    new_comment3=new_comment3.lower()                                                                                       
    #Step6:去除标点符号和数字
    new_comment4=exclude_word(new_comment3)
    comment_new=str.split(new_comment4,'¥')
    #Step7:特征处理
    test_tf_idf =tf_idf.transform(comment_new)
    x=test_tf_idf
    x=Standard_demo(x)
    prediction=vote.predict(x)
    return prediction

In [385]:
prediction=test_demo()

In [386]:
testdata1['prediction']=list(prediction)[0:-1]
testdata1=testdata1[testdata1['prediction']==1]

In [391]:
testdata1=testdata1.iloc[:,[1,3]].set_index('id')

In [392]:
testdata1.to_csv('testdata1_new_complain.csv')